In [1]:
import praw
import pandas as pd
from datetime import datetime
import json

# Create a Reddit instance (Replace with your credentials)
reddit = praw.Reddit(
    client_id='pC9dZB_wqZ9Son7HPJUc8w',
    client_secret='4hpR2C7Fpov0QGM7g5QNKJqsN0SmRA',
    user_agent='narative by /u/No_Type7179'
)

# Function to test Reddit API access
def test_reddit_access():
    try:
        subreddit = reddit.subreddit('all')
        print(f"Accessing subreddit: {subreddit.display_name}")
    except Exception as e:
        print(f"Error: {e}")
        return False
    return True

# Function to fetch top posts based on search query
def fetch_top_posts(search_query, subreddit_name='all', limit=10):
    try:
        subreddit = reddit.subreddit(subreddit_name)
        top_posts = subreddit.search(search_query, sort='top', limit=limit)

        post_data = []
        for post in top_posts:
            post_datetime = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            upvotes = post.ups
            downvotes = upvotes - post.score

            awards = [award['name'] for award in getattr(post, 'all_awards', [])]
            flair = getattr(post, 'link_flair_text', 'No Flair')

            post_data.append({
                'Title': post.title,
                'Body/Content': post.selftext,
                'Post ID': post.id,
                'Subreddit': post.subreddit.display_name,
                'Date/Time': post_datetime,
                'Upvotes': upvotes,
                'Downvotes': downvotes,
                'Number of Comments': post.num_comments,
                'Link': post.url,
                'Awards': awards,
                'Flair': flair
            })

        if not post_data:
            print("No posts found for the given query.")
            return

        df = pd.DataFrame(post_data)

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        json_filename = f'reddit_top_posts_{search_query}_{timestamp}.json'
        with open(json_filename, 'w') as f:
            json.dump(post_data, f, indent=4)

        print(df.head())  # Show only the first few rows for large data
        print(f"Data saved to {json_filename}")

    except Exception as e:
        print(f"Error fetching posts: {e}")

# Main function
def main():
    if test_reddit_access():
        search_query = input("Enter the topic you want to search for: ")
        subreddit_name = input("Enter the subreddit to search in (default is 'all'): ") or 'all'
        try:
            limit = int(input("Enter the number of posts to fetch (default is 10): ") or 10)
        except ValueError:
            print("Invalid input for the number of posts. Using default value of 10.")
            limit = 10

        fetch_top_posts(search_query, subreddit_name, limit)

if __name__ == "__main__":
    main()


Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


Accessing subreddit: all
                                               Title  \
0  Megathread: House Votes to Impeach President D...   
1                                           Hell nah   
2                                   Oh hell yeah man   

                                        Body/Content Post ID Subreddit  \
0  The United States House of Representatives has...  ecm1zg  politics   
1                                                     m4qzhr     memes   
2                                                     hu8n0z     memes   

             Date/Time  Upvotes  Downvotes  Number of Comments  \
0  2019-12-19 01:36:13   147737          0               50756   
1  2021-03-14 08:00:52   147702          0                1119   
2  2020-07-19 21:33:44   143628          0                 655   

                                                Link Awards       Flair  
0  https://www.reddit.com/r/politics/comments/ecm...     []  Megathread  
1                https://i.redd.it/ecdmc